가장 중요한 목표는 다음과 같습니다.

1. 로그 보강: VSCode의 출력 한계를 극복하고, 모든 성공/실패 기록을 파일로 남깁니다.

2. 안정적인 스크롤: 스크롤 크기를 줄이고, 스크롤 횟수를 로그로 기록하여 동작을 추적합니다.

3. 고유 키 활용: key를 모든 로그 메시지와 에러 스크린샷에 포함시켜 누락된 데이터를 정확하게 파악할 수 있도록 합니다.

4. time.sleep() 최소화: WebDriverWait을 사용하여 로딩을 더 안정적으로 처리합니다.

In [ ]:
import os
import time
import pandas as pd
import logging
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoAlertPresentException, TimeoutException, StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.chrome.options import Options

# ---------------------- 로그 설정 ----------------------
log_file_name = f"crawling_log_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    filename=log_file_name,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    encoding='utf-8'
)
console_handler = logging.StreamHandler()
console_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
logging.getLogger().addHandler(console_handler)

DOWNLOAD_DIR = r"C:\Users\mstel\Downloads"
options = Options()
options.add_experimental_option("detach", True)
options.add_experimental_option("prefs", {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True
})

driver = webdriver.Chrome(options=options)
driver.get("https://tmacs.kotsa.or.kr/web/TG/TG300/TG3100/Tg2127.jsp?mid=S1810")
wait = WebDriverWait(driver, 15)

# ---------------------- 팝업 / 다운로드 ----------------------
def wait_for_download_complete(download_dir=DOWNLOAD_DIR, timeout=30):
    before = set(os.listdir(download_dir))
    end_time = time.time() + timeout
    while time.time() < end_time:
        after = set(os.listdir(download_dir))
        new_files = after - before
        if new_files and any(f.endswith((".xlsx", ".xls")) for f in new_files):
            for f in new_files:
                if (f.endswith(".xlsx") or f.endswith(".xls")) and not f.startswith("~"):
                    return f
        time.sleep(1)
    return None

def handle_structure_download(driver, wait, key):
    main_window = driver.current_window_handle
    retries = 0
    while retries < 2:
        try:
            time.sleep(1)
            for handle in driver.window_handles:
                if handle != main_window:
                    driver.switch_to.window(handle)
                    break
            
            tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#tabId03")))
            tab.click()
            time.sleep(5)

            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "dl.jogun dd")))
            dd_elements = driver.find_elements(By.CSS_SELECTOR, "dl.jogun dd")
            사고지역 = dd_elements[0].text.strip() if len(dd_elements) > 0 else ""
            사고장소 = dd_elements[1].text.strip() if len(dd_elements) > 1 else ""
            조회기간 = dd_elements[2].text.strip() if len(dd_elements) > 2 else ""

            excel_btn = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "#new_popup > div.pop_cont > div.btn_box > a.btn.exbtn")
            ))
            excel_btn.click()

            downloaded_file = wait_for_download_complete()
            if downloaded_file:
                downloaded_path = os.path.join(DOWNLOAD_DIR, downloaded_file)
                last_size = -1
                for _ in range(5):
                    if os.path.exists(downloaded_path):
                        current_size = os.path.getsize(downloaded_path)
                        if current_size > 0 and current_size == last_size:
                            break
                        last_size = current_size
                    time.sleep(1)

                df = pd.read_excel(downloaded_path, header=[0, 1, 2])
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = ['_'.join([str(c).strip() for c in col if str(c).strip() not in ('nan', '')]) for col in df.columns]
                else:
                    df.columns = [str(c).strip() for c in df.columns]
                df.columns = [col.strip() for col in df.columns]
                df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
                df.insert(0, '사고지역', 사고지역)
                df.insert(1, '사고장소', 사고장소)
                df.insert(2, '조회기간', 조회기간)
                logging.info(f"[✅ 성공] 데이터 수집 완료 (key: {key})")
                try:
                    os.remove(downloaded_path)
                except OSError as e:
                    logging.error(f"[❌ 삭제 실패] 다운로드된 파일 삭제 실패: {e}")
                return df
            else:
                logging.warning(f"[⚠️ 타임아웃] 파일 다운로드 실패 (key: {key})")
                retries += 1
                logging.warning(f"[⚠️ 재시도] 다운로드 실패 (key: {key}) (재시도 {retries}/2)")
                if len(driver.window_handles) > 1:
                    try:
                        driver.close()
                        driver.switch_to.window(main_window)
                    except Exception:
                        pass
                continue
        except Exception as e:
            retries += 1
            logging.error(f"[❌ 실패] 다운로드 처리 중 오류 발생 (key: {key}) - {e} (재시도 {retries}/2)")
            driver.save_screenshot(f"error_{key}.png")
            time.sleep(1)
        finally:
            if len(driver.window_handles) > 1:
                try:
                    driver.close()
                except Exception:
                    pass
            driver.switch_to.window(main_window) # <--- 이 부분이 핵심
    
    if retries == 2:
        logging.error(f"[⛔ 완전 실패] 다운로드 처리 완전 실패 (key: {key})")
    return None

# ---------------------- visited ----------------------
visited = set()

# ---------------------- 버튼 클릭 + 내부 스크롤 ----------------------
def download_page(driver, wait, year_value, sido_text, jijace_text):
    scroll_element = driver.find_element(By.CSS_SELECTOR, ".rMateH5__VBrowserScrollBar")
    last_scroll_height = -1
    scroll_count = 0
    
    current_page_dfs = []

    while True:
        buttons = driver.find_elements(By.CSS_SELECTOR, "#rMateH5__Content69 > div > span > img")
        
        if not buttons:
            logging.info(f"[{sido_text}_{jijace_text}] 버튼 없음 → 종료")
            break

        logging.info(f"[{sido_text}_{jijace_text}] 스크롤 {scroll_count}회차 시작. 현재 DOM에 있는 버튼 수: {len(buttons)}개.")

        for idx, btn in enumerate(buttons):
            key = f"{year_value}_{sido_text}_{jijace_text}_scroll{scroll_count}_idx{idx}"

            if key in visited:
                continue

            try:
                button_cell = btn.find_element(By.XPATH, "./ancestor::div[contains(@class, 'rMateH5__ImageItemRenderer')]")
                parent_container = button_cell.find_element(By.XPATH, "./parent::div")
                branch_name = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn25").text.strip()
                incident_count = parent_container.find_element(By.CSS_SELECTOR, "span.rMateH5__DataGridColumn27").text.strip()
            except StaleElementReferenceException:
                logging.warning(f"[{sido_text}_{jijace_text}] StaleElementReferenceException 발생. 루프 재시작.")
                buttons = driver.find_elements(By.CSS_SELECTOR, "#rMateH5__Content69 > div > span > img")
                continue
            except Exception as e:
                logging.error(f"[{sido_text}_{jijace_text}] [❌ 데이터 파싱 실패] {e} (key: {key})")
                continue

            if branch_name == '합계':
                logging.info(f"[{sido_text}_{jijace_text}] 🚫 건너뛰기: '{branch_name}' 항목은 처리하지 않습니다.")
                visited.add(key)
                continue

            logging.info(f"[{sido_text}_{jijace_text}] 👉 클릭: '{branch_name}' (사고수: {incident_count}) (key: {key})")

            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(1)

            df = handle_structure_download(driver, wait, key)
            if df is not None:
                current_page_dfs.append(df)
                visited.add(key)
            else:
                visited.add(key)
        
        current_scroll = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
        max_scroll = driver.execute_script("return arguments[0].scrollHeight - arguments[0].clientHeight;", scroll_element)
        
        if current_scroll >= max_scroll:
            logging.info(f"[{sido_text}_{jijace_text}] 📌 스크롤 끝. 현재까지 총 {len(current_page_dfs)}건 수집 완료.")
            break

        if current_scroll == last_scroll_height: 
            logging.warning(f"[{sido_text}_{jijace_text}] 스크롤 위치 변화 없음. 추가 콘텐츠 로딩 대기 시도. (3초)")
            time.sleep(3)
            current_scroll_after_wait = driver.execute_script("return arguments[0].scrollTop;", scroll_element)
            if current_scroll_after_wait == last_scroll_height:
                logging.error(f"[{sido_text}_{jijace_text}] ⛔️ 스크롤 위치 변화 없음 및 추가 대기 후에도 동일 → 현재 지자체 처리 강제 종료")
                break
        
        last_scroll_height = current_scroll
        scroll_count += 1
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollTop + 500;", scroll_element)
        logging.info(f"[{sido_text}_{jijace_text}] 스크롤 다운...")
        time.sleep(1)

    logging.info(f"[{sido_text}_{jijace_text}] ✅ 이 지자체의 모든 버튼 다운로드 완료")
    return current_page_dfs

# ---------------------- 전체 다운로드 ----------------------
def download_all(driver, wait, year_value, sido_text, jijace_text):
    return download_page(driver, wait, year_value, sido_text, jijace_text)

# ---------------------- 실행 ----------------------
test_years = ["2023"]
all_collected_dfs = []
try:
    for year_value in test_years:
        logging.info(f"\n--- {year_value}년도 데이터 수집 시작 ---")
        year_select = Select(driver.find_element(By.ID, "Year"))
        year_select.select_by_value(year_value)

        sido_options = driver.find_elements(By.CSS_SELECTOR, "#sido option")
        for sido_index in range(1, 2):
            sido_select = Select(driver.find_element(By.ID, "sido"))
            sido_select.select_by_index(sido_index)
            sido_text = sido_select.first_selected_option.text
            
            logging.info(f"\n--- {year_value}년 {sido_text} 데이터 수집 시작 ---")
            
            wait.until(EC.presence_of_element_located((By.ID, "jijace")))
            jijace_select = Select(driver.find_element(By.ID, "jijace"))
            jijace_select.select_by_index(0)
            jijace_text = jijace_select.first_selected_option.text
            time.sleep(0.5)

            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.btn_wrap > a")))
            search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.btn_wrap > a")))
            driver.execute_script("arguments[0].click();", search_button)

            try:
                alert = driver.switch_to.alert
                alert.accept()
            except NoAlertPresentException:
                pass

            time.sleep(3)
            jijace_dfs = download_all(driver, wait, year_value, sido_text, jijace_text)
            if jijace_dfs:
                all_collected_dfs.extend(jijace_dfs)
        
        logging.info(f"--- {year_value}년도 데이터 수집 완료 ---")
        time.sleep(5)

finally:
    driver.quit()

# ---------------------- 수집된 DF 확인 및 처리 ----------------------
logging.info("\n--- 스크래핑 후 데이터 처리 시작 ---")

if all_collected_dfs:
    logging.info(f"총 수집된 DataFrame 개수: {len(all_collected_dfs)}")
    final_df = pd.concat(all_collected_dfs, ignore_index=True)
    logging.info(f"모든 DataFrame을 합친 최종 DataFrame의 총 행 수: {len(final_df)}")
    
    initial_rows = len(final_df)
    final_df.drop_duplicates(inplace=True)
    rows_after_dedup = len(final_df)
    logging.info(f"중복 제거 전 행 수: {initial_rows}")
    logging.info(f"중복 제거 후 행 수: {rows_after_dedup}")
    logging.info(f"제거된 중복 행 수: {initial_rows - rows_after_dedup}")

    output_excel_path = os.path.join(DOWNLOAD_DIR, "사고데이터_통합본.xlsx")
    final_df.to_excel(output_excel_path, index=False)
    logging.info(f"✅ 최종 데이터가 '{output_excel_path}' 파일로 저장되었습니다.")
else:
    logging.info("수집된 DataFrame이 없어 최종 처리할 데이터가 없습니다.")

logging.info("\n--- 스크래핑 및 데이터 처리 완료 ---")

2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:26,837 - INFO - 
--- 2023년도 데이터 수집 시작 ---
2025-08-07 01:18:27,329 - INFO - 
--- 2023년 서울 데이터 수집 시작 ---
2025-08-07 01:18:27,329 - INFO - 
--- 2023년 서울 데이터 수집 시작 ---
2025-08-07 01:18:27,329 - INFO - 
--- 2023년 서울 데이터 수집 시작 ---
2025-08-07 01:18:27,329 - INFO - 
--- 2023년 서울 데이터

In [ ]:
all_collected_dfs